In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN2005"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10


wavelet_levels = 6    #number of wavelet layer to have in the network
convolution_levels = 5    #number of convolution to have in the network
pooling_stride = [2, 2, 2, 2, 2]    #stride of pooling in convolution
weights_shape = [[5, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
granularity = np.prod(pooling_stride)
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [7]:
print pos_weight

3.18601822124


In [9]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [10]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [11]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [12]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation)
                        

In [13]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size)

In [14]:
trainer.train(max_iter = 10000,
                train_eval_step = 100,
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.247068, fmeasure_max= 0.355812, precision= 0.216406, recall= 1.000000
Train Random guess:  , auc= 0.226076fmeasure_max= 0.357041, precision= 0.222321, recall= 0.906137
Validation Iter 0, auc= 0.218163, fmeasure_max= 0.390141, precision= 0.245459, recall= 0.950257
Validation Random guess:  auc= 0.234313, fmeasure_max= 0.372183, precision= 0.229092, recall= 0.991424
Minibatch Loss= 1.037257
Tarin Iter 100, auc= 0.585328, fmeasure_max= 0.547995, precision= 0.435328, recall= 0.739344
Train Random guess:  , auc= 0.264451fmeasure_max= 0.385321, precision= 0.238730, recall= 0.998361
Validation Iter 100, auc= 0.618352, fmeasure_max= 0.593516, precision= 0.466210, recall= 0.816467
Validation Random guess:  auc= 0.233138, fmeasure_max= 0.372078, precision= 0.228740, recall= 0.996569
Minibatch Loss= 0.841286
Tarin Iter 200, auc= 0.766217, fmeasure_max= 0.756897, precision= 0.701278, recall= 0.822097
Train Random guess:  , auc= 0.200711fmeasure_max= 0.345519, precision= 0.2088

In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [15]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.923566 F1_max = 0.847661 prec = 0.836983 rec = 0.858615
Test Random guess: auc= 0.229948 fmeasure_max= 0.375876 , precision= 0.231624 , recall= 0.996453


In [16]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [17]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})